In [1]:
from typing import List, Tuple

In [2]:
import numpy as np

In [3]:
def med3(u: float, v: float, w: float) -> float:
    if (u <= v and v <= w) or (u >= v and v >= w):
        return v
    elif (u <= w and w <= v) or (u >= w and w >= v):
        return w
    else:
        return u


def imed3(u: float, v: float, w: float) -> int:
    if (u <= v and v <= w) or (u >= v and v >= w):
        return 0
    elif (u <= w and w <= v) or (u >= w and w >= v):
        return 1
    else:
        return -1


def sm_3(
    x: List[float], y: List[float], n: int, end_rule: int
) -> Tuple[bool, List[float], List[float]]:
    """
    y[] := Running Median of three (x) = "3 (x[])" with "copy ends"
     ---  return chg := ( y != x ) */
    """
    chg = False

    if n <= 2:
        for i in range(n):
            y[i] = x[i]
        return False

    for i in range(1, n - 1):
        j = imed3(x[i - 1], x[i], x[i + 1])
        y[i] = x[i + j]
        chg = chg or j

    chg, y, x = sm_DO_ENDRULE(y, x, n, chg, end_rule)

    return chg, x, y


def sm_DO_ENDRULE(
    y: List[float], x: List[float], n: int, chg: bool, end_rule: int
) -> Tuple[bool, List[float], List[float]]:
    if end_rule == 0: #"sm_NO_ENDRULE"
        pass
    elif end_rule == 1: #"sm_COPY_ENDRULE"
        y[0] = x[0]
        y[n-1] = x[n-1]
    elif end_rule == 2: #"sm_TUKEY_ENDRULE"
        y[0] = np.median([3 * y[1] - 2 * y[2], x[0], y[1]])
        chg = chg or (y[0] != x[0])
        y[n-1] = np.median([y[n-2], x[n-1], 3 * y[n-2] - 2 * y[n-3]])
        chg = chg or (y[n-1] != x[n-1])
    else:
        raise RuntimeError(f"invalid end-rule for running median of 3: {end_rule}")

    return chg, y, x


def sm_3R(
    x: float,
    y: float,
    z: float,
    n: int,
    end_rule: int,
) -> Tuple[int, List[float], List[float]]:

    chg, y, x = sm_3(x, y, n, 1) #"sm_COPY_ENDRULE"
    it = chg

    while chg:
        chg, y, z = sm_3(y, z, n, 0) #"sm_NO_ENDRULE"
        if chg:
            it += 1
            for i in range(1, n - 1):
                y[i] = z[i]

    if n > 2:
        chg, y, x = sm_DO_ENDRULE(y, x, n, chg, end_rule)

    if it:
        return it, x, y, z
    else:
        return int(chg), x, y, z


def sptest(x: List[float], i: int) -> bool:
    """
    Split test:
       Are we at a /-\ or \_/ location => split should be made ?
    """
    if x[i] != x[i + 1]:
        return False
    elif (x[i - 1] <= x[i] and x[i + 1] <= x[i + 2]) or (
        x[i - 1] >= x[i] and x[i + 1] >= x[i + 2]
    ):
        return False
    else:
        return True


def sm_split3(
    x: List[float], y: List[float], n: int, do_ends: bool
) -> Tuple[bool, List[float], List[float]]:

    # y[] := S(x[])  where S() = "sm_split3"
    chg = False

    for i in range(n):
        y[i] = x[i]

    if n <= 4:
        return False, y, x

    # Colin Goodall doesn't do splits near ends
    # in spl() in Statlib's "smoother" code !!
    if do_ends and sptest(x, 1):
        chg = True
        y[1] = x[0]
        y[2] = med3(x[2], x[3], 3 * x[3] - 2 * x[4])

    for i in range(2, n - 3):
        if sptest(x, i):
            # plateau at x[i] == x[i+1]
            # at left :
            if -1 < (j := imed3(x[i], x[i - 1], 3 * x[i - 1] - 2 * x[i - 2])):
                if j == 0:
                    y[i] = x[i - 1]
                else:
                    y[i] = 3 * x[i - 1] - 2 * x[i - 2]
                chg = y[i] != x[i]
            # at right :
            if -1 < (j := imed3(x[i + 1], x[i + 2], 3 * x[i + 2] - 2 * x[i + 3])):
                if j == 0:
                    y[i + 1] = x[i + 2]
                else:
                    y[i + 1] = 3 * x[i + 2] - 2 * x[i + 3]
                chg = y[i + 1] != x[i + 1]
    if do_ends and sptest(x, n - 3):
        chg = True
        y[n - 2] = x[n - 1]
        y[n - 3] = med3(x[n - 3], x[n - 4], 3 * x[n - 4] - 2 * x[n - 5])
    return chg, x, y


def sm_3RS3R(
    x: List[float],
    y: List[float],
    z: List[float],
    w: List[float],
    n: int,
    end_rule: int,
    split_ends: bool,
) -> Tuple[int, List[float], List[float], List[float], List[float]]:
    # y[1:n] := "3R S 3R"(x[1:n]);  z = "work";

    it, x, y, z = sm_3R(x, y, z, n, end_rule)
    chg, y, z = sm_split3(y, z, n, split_ends)
    if chg:
        (
            it2,
            z,
            y,
            w,
        ) = sm_3R(z, y, w, n, end_rule)
        it += it2
    # else y == z already
    return (it + chg), x, y, z, w


def sm_3RSS(
    x: List[float],
    y: List[float],
    z: List[float],
    n: int,
    end_rule: int,
    split_ends: bool,
) -> Tuple[int, List[float], List[float], List[float]]:
    # y[1:n] := "3RSS"(x[1:n]);  z = "work"

    it, x, y, z = sm_3R(x, y, z, n, end_rule)
    chg, y, z = sm_split3(y, z, n, split_ends)
    if chg is True:
        z, y = sm_split3(z, y, n, split_ends)
    # else  y == z already
    return (iter + (chg)), x, y, z


def sm_3RSR(
    x: List[float],
    y: List[float],
    z: List[float],
    w: List[float],
    n: int,
    end_rule: int,
    split_ends: bool,
) -> Tuple[int, List[float], List[float], List[float], List[float]]:

    # y[1:n] := "3RSR"(x[1:n]);  z := residuals; w = "work";

    # == "SR" (as follows) is stupid ! (MM) ==

    it, x, y, z = sm_3R(x, y, z, n, end_rule)

    while chg is True:
        it += 1
        chg, y, z = sm_split3(y, z, n, split_ends)
        ch2, z, y, w = sm_3R(z, y, w, n, end_rule)
        chg = chg or ch2

        if chg is False:
            break
        if iter > 2 * n:
            break  # INF.LOOP stopper
        for i in range(n):
            z[i] = x[i] - y[i]

    return it, x, y, z, w

In [8]:
x1 = [4, 1, 3, 6, 6, 4, 1, 6, 2, 4, 2]
n = len(x1)
y = np.zeros(n)
z = np.zeros(n)
w = np.zeros(n)
end_rule = 0
split_ends = True

In [5]:
it, x, y = sm_3(x=x1, y=y, n=n, end_rule=2)

In [6]:
it

1

In [7]:
y

array([4., 3., 3., 6., 6., 4., 4., 2., 4., 2., 2.])

In [5]:
it, x, y, z = sm_3R(x=x1, y=y, z=z, n=n, end_rule=2)

In [6]:
y

[3.0, 3.0, 3.0, 6.0, 6.0, 4.0, 4.0, 4.0, 2.0, 2.0, 2.0]

In [9]:
iend = 2
#se = True if iend < 0 else False
it, x, y, z, w = sm_3RS3R(x=x1, y=y, z=z, w=w, n=n, end_rule=abs(2), split_ends=split_ends)

In [10]:
y

array([3., 3., 3., 3., 4., 4., 4., 4., 2., 2., 2.])

In [11]:
target = np.array([3, 3, 3, 3, 4, 4, 4, 4, 2, 2, 2])

In [13]:
all(y == target)

True

In [197]:
from copy import deepcopy

In [222]:
y1 = deepcopy(y)

In [223]:
for i in range(1, n - 1):
    j = imed3(x[i - 1], x[i], x[i + 1])
    y[i] = x[i + j]
    chg = chg or j

print(y)

[0. 3. 3. 6. 6. 4. 4. 2. 4. 2. 0.]


In [213]:
y[0] = med3(3 * y[1] - 2 * y[2], x[0], y[1])

In [214]:
y

array([3., 3., 3., 6., 6., 4., 4., 2., 4., 2., 0.])

In [215]:
chg = chg or (y[0] != x[0])

In [217]:
y[-1] = med3(y[n - 2], x[n - 1], 3 * y[n - 2] - 2 * y[n - 3])

In [218]:
chg = chg or (y[n - 1] != x[n - 1])

In [219]:
chg

1

In [220]:
y

array([3., 3., 3., 6., 6., 4., 4., 2., 4., 2., 2.])

In [224]:
y[0] = med3(3 * y[1] - 2 * y[2], x[0], y[1])
chg = chg or (y[0] != x[0])
y[n - 1] = med3(y[n - 2], x[n - 1], 3 * y[n - 2] - 2 * y[n - 3])
chg = chg or (y[n - 1] != x[n - 1])

In [225]:
y

array([3., 3., 3., 6., 6., 4., 4., 2., 4., 2., 2.])

In [180]:
i = 1
j = imed3(x[i - 1], x[i], x[i + 1])

In [191]:
for i in range(0, len(x1)):
    print(imed3(x1[i-1], x1[i], x1[i+1]))

-1
1
0
0
0
0
-1
1
1
1


IndexError: list index out of range

In [163]:
print([x1[_-1] for _, j in enumerate(x1)])

[2, 4, 1, 3, 6, 6, 4, 1, 6, 2, 4]


In [11]:
from enum import IntEnum

class endrule(IntEnum):
    none = 0
    copy = 1
    tukey = 2

In [12]:
a = endrule(2)

In [13]:
a.name

'tukey'

In [17]:
b = endrule(a - 1)

In [18]:
b.name

'copy'